# "2019 January One Day Holding Long Short Portfolio BackTest"
> "2019 January Portfolio"
- toc: false
- branch: master
- badges: false
- comments: true
- categories: [long, short, longshort]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

In [2]:
#hide_input
import numpy as np
import pandas as pd
import seaborn as sns
import os
import time
import gc
import glob
import matplotlib.pyplot as plt

In [4]:
#hide_input
df_testp=pd.read_csv('D:\\Downloads\\20194dayreturnpred2019train60K.csv')
df_test=pd.read_csv('D:\\Downloads\\sample28K20194dayreturnpred2019train60K.csv')
df_tes=pd.read_csv('D:\\Downloads\\10Q20194dayreturnpred2019train60K.csv')

In [5]:
#hide
df_testp=df_testp.loc[df_testp.label!='nan'].loc[df_testp.Date>=20190101].loc[df_testp.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_test=df_test.loc[df_test.label!='nan'].loc[df_test.Date>=20190101].loc[df_test.Date<20190201].sort_values(by='Date',ascending=True).copy()
df_tes=df_tes.loc[df_tes.label!='nan'].loc[df_tes.Date>=20190101].loc[df_tes.Date<20190201].sort_values(by='Date',ascending=True).copy()
#df_testp.reset_index(drop=True, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


**Long-Short Portfolio**

In [9]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])
    
#hide
df_ag=df_test[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [10]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-01-04,-1.0,-3.755620,1.037556,1.007010,1.004190,-0.669154,0.965937
1,2019-01-04,-1.0,-3.755620,1.037556,1.007010,1.004190,-0.851344,0.965937
2,2019-01-11,-1.0,-1.868240,0.928826,1.020012,1.020342,-0.691317,0.953611
3,2019-01-11,-1.0,-1.868240,0.928826,1.020012,1.020342,-0.731021,0.953611
4,2019-01-11,1.0,-2.027027,0.909999,1.020012,1.020342,-0.691317,0.953611
5,2019-01-11,1.0,-2.027027,0.909999,1.020012,1.020342,-0.731021,0.953611
6,2019-01-14,-1.0,-0.561798,0.915111,1.030949,1.026989,-1.474983,1.074158
7,2019-01-14,-1.0,-0.561798,0.915111,1.030949,1.026989,-1.323502,1.074158
8,2019-01-14,1.0,2.314854,0.936295,1.030949,1.026989,-1.474983,1.074158
9,2019-01-14,1.0,2.314854,0.936295,1.030949,1.026989,-1.323502,1.074158


In [14]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)

**Long portfolio**

In [15]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])

df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [16]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [17]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-01-30,1.0,-1.261737,0.933635,1.041537,1.041829,10.898869,1.037559
1,2019-01-31,1.0,0.748486,0.940623,1.041537,1.041829,9.951025,1.050766


In [18]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)

**Short Portfolio**

In [22]:
#hide
df_agg=df_testp[['Date','pred','labelr1']].loc[df_testp.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_agg['cumreturn1']=0.0
for l in range(len(df_agg)):
    df_agg['cumreturn1'][l]=np.prod((1+(df_agg['labelr1']*df_agg['pred']/100))[:l+1])

df_ag=df_test[['Date','pred','labelr1']].loc[df_test.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_ag['cumreturn1a']=0.0
for l in range(len(df_ag)):
    df_ag['cumreturn1a'][l]=np.prod((1+(df_ag['labelr1']*df_ag['pred']/100))[:l+1])
    
df_a=df_tes[['Date','pred','labelr1']].loc[df_tes.pred==-1.0].groupby(['Date','pred']).sum().reset_index();
df_a['cumreturn1b']=0.0
for l in range(len(df_a)):
    df_a['cumreturn1b'][l]=np.prod((1+(df_a['labelr1']*df_a['pred']/100))[:l+1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [23]:
#hide
if len(df_agg)>0:
    import altair as alt
    import datetime
    df_agg['Date']=df_agg['Date'].astype(str).str[:4]+'-'+df_agg['Date'].astype(str).str[4:6]+'-'+df_agg['Date'].astype(str).str[6:]
    df_agg['Date']=df_agg['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=pd.read_csv('D:\\Downloads\\GSPC.csv')
    df_dow['Date']=df_dow['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_dow=df_dow.loc[df_dow.Date>=df_agg.Date.min()].loc[df_dow.Date<=df_agg.Date.max()].reset_index()
    df_dow['return']=((df_dow['Close'].shift(-1)-df_dow['Close'])/df_dow["Close"])
    df_dow['cumreturndow']=0.0
    for l in range(len(df_dow)):
        df_dow['cumreturndow'][l]=np.prod((1+(df_dow['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_dow[['Date','cumreturndow']], right_on='Date', left_on='Date',how='inner')
    df_sp500=pd.read_csv('D:\\Downloads\\DJI.csv')
    df_sp500['Date']=df_sp500['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_sp500=df_sp500.loc[df_sp500.Date>=df_agg.Date.min()].loc[df_sp500.Date<=df_agg.Date.max()].reset_index()
    df_sp500['return']=((df_sp500['Close'].shift(-1)-df_sp500['Close'])/df_sp500["Close"])
    df_sp500['cumreturnsp500']=0.0
    for l in range(len(df_sp500)):
        df_sp500['cumreturnsp500'][l]=np.prod((1+(df_sp500['return']))[:l+1])
    df_agg=pd.merge(df_agg,df_sp500[['Date','cumreturnsp500']], right_on='Date', left_on='Date',how='inner')
    df_ag['Date']=df_ag['Date'].astype(str).str[:4]+'-'+df_ag['Date'].astype(str).str[4:6]+'-'+df_ag['Date'].astype(str).str[6:]
    df_ag['Date']=df_ag['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_ag[['Date','cumreturn1a']], right_on='Date', left_on='Date',how='inner')
    df_a['Date']=df_a['Date'].astype(str).str[:4]+'-'+df_a['Date'].astype(str).str[4:6]+'-'+df_a['Date'].astype(str).str[6:]
    df_a['Date']=df_a['Date'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
    df_agg=pd.merge(df_agg,df_a[['Date','cumreturn1b']], right_on='Date', left_on='Date',how='inner')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [24]:
#hide_input
df_agg

,Date,pred,labelr1,cumreturn1,cumreturndow,cumreturnsp500,cumreturn1a,cumreturn1b
0,2019-01-04,-1.0,-3.755620,1.037556,1.007010,1.004190,-0.520401,0.952680
1,2019-01-11,-1.0,-1.868240,0.928826,1.020012,1.020342,-0.208970,0.922115
2,2019-01-14,-1.0,-0.561798,0.934044,1.030949,1.026989,-0.421640,1.038681
3,2019-01-25,-1.0,-0.476167,0.750710,1.044199,1.046731,0.216666,1.424472
4,2019-01-28,-1.0,3.804956,0.722146,1.042679,1.048939,0.085174,1.363974
5,2019-01-29,-1.0,-5.349998,0.760781,1.058892,1.067498,0.025088,1.273995
6,2019-01-30,-1.0,-1.975703,0.775812,1.067995,1.066850,0.000871,1.522626
7,2019-01-31,-1.0,-2.537389,0.795497,1.067995,1.066850,0.000866,1.502078


In [25]:
#hide_input
import altair as alt
a1 = alt.Chart(df_agg).mark_trail(color='steelblue').encode(
    x='Date:T',
    y='cumreturn1:Q',
    size='cumreturn1',
    
).properties(
    width=1000,
    height=1000,
    title='10KPortfolio (blue) vs 8KPortfolio (brown) vs 10QPortfolio (yellow) vs  Dow (black) vs SP500 (red)'
)

a2 = alt.Chart(df_agg).mark_circle(color='black',size=40).encode(
    x='Date:T',
    y='cumreturndow:Q',

).properties(
    width=1000,
    height=1000
)

a3 = alt.Chart(df_agg).mark_tick(color='red').encode(
    x='Date:T',
    y='cumreturnsp500:Q',

).properties(
    width=1000,
    height=1000
)

a4 = alt.Chart(df_agg).mark_line(color='brown').encode(
    x='Date:T',
    y='cumreturn1a:Q',

).properties(
    width=1000,
    height=1000
)

a5 = alt.Chart(df_agg).mark_line(color='yellow').encode(
    x='Date:T',
    y='cumreturn1b:Q',

).properties(
    width=1000,
    height=1000
)

a1 + a2 + a3 + a4 + a5

alt.LayerChart(...)